In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 

import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD

In [2]:
warnings.filterwarnings("ignore")

In [3]:
combine_df = pd.read_pickle("./combine_df_leson1.pkl")

In [4]:
combine_df.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


**1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**
- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'. 
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


In [5]:
count_vectorizer_stemmed = CountVectorizer(max_df=0.9, max_features = 1000, stop_words='english', binary=False)

bag_of_words_stemmed = count_vectorizer_stemmed.fit_transform(combine_df.tweet_stemmed.apply(lambda x: ' '.join([word for word in x])))

feature_names = count_vectorizer_stemmed.get_feature_names()
df_bag_of_words_stemmed = pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names)
df_bag_of_words_stemmed.head()

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_bag_of_words_stemmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Columns: 1000 entries, abl to yummi
dtypes: int64(1000)
memory usage: 375.1 MB


In [7]:
count_vectorizer_lemmatized= CountVectorizer(max_df=0.9, max_features = 1000, stop_words='english', binary=False)

bag_of_words_lemmatized = count_vectorizer_lemmatized.fit_transform(combine_df.tweet_lemmatized.apply(lambda x: ' '.join([word for word in x])))

feature_names = count_vectorizer_lemmatized.get_feature_names()
df_bag_of_words_lemmatized = pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names)
df_bag_of_words_lemmatized.head()

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_bag_of_words_lemmatized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Columns: 1000 entries, able to yummy
dtypes: int64(1000)
memory usage: 375.1 MB


**2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**
- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.
- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [9]:
tfidf_vectorizer_stemmed = TfidfVectorizer(max_df=0.9, max_features = 1000, stop_words='english')

values_stemmed = tfidf_vectorizer_stemmed.fit_transform(combine_df.tweet_stemmed.apply(lambda x: ' '.join([word for word in x])))

feature_names = tfidf_vectorizer_stemmed.get_feature_names()
df_tfidf_vectorizer_stemmed = pd.DataFrame(values_stemmed.toarray(), columns = feature_names)
df_tfidf_vectorizer_stemmed.head()

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df_tfidf_vectorizer_stemmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Columns: 1000 entries, abl to yummi
dtypes: float64(1000)
memory usage: 375.1 MB


In [11]:
tfidf_vectorizer_lemmatized = TfidfVectorizer(max_df=0.9, max_features = 1000, stop_words='english')

values_lemmatized = tfidf_vectorizer_lemmatized.fit_transform(combine_df.tweet_lemmatized.apply(lambda x: ' '.join([word for word in x])))

feature_names = tfidf_vectorizer_lemmatized.get_feature_names()
df_tfidf_vectorizer_lemmatized = pd.DataFrame(values_lemmatized.toarray(), columns = feature_names)
df_tfidf_vectorizer_lemmatized.head()

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_tfidf_vectorizer_lemmatized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Columns: 1000 entries, able to yummy
dtypes: float64(1000)
memory usage: 375.1 MB


**3. Проверьте ваши векторайзеры на корпусе, который использовали на вебинаре, составьте таблицу: метод векторизации и скор, который вы получили (в методах векторизации поизменяйте параметры чтобы добиться лучшего скора), обратите внимание как падает/растёт скор при уменьшении количества фичей и изменении параметров, также попробуйте применить к векторайзерам PCA для сокращения размерности посмотрите на качество сделайте выводы**

In [13]:
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [14]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

In [15]:
encoder = preprocessing.LabelEncoder()

train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [16]:
count_vect_corp = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
tfidf_vec_corp = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect_corp_mod = CountVectorizer(max_df=0.9, max_features = 1000, stop_words='english', analyzer='word', token_pattern=r'\w{1,}')
tfidf_vec_corp_mod = TfidfVectorizer(max_df=0.9, max_features = 1000, stop_words='english', analyzer='word', token_pattern=r'\w{1,}')

In [17]:
count_vect_corp.fit(trainDF['text'])
tfidf_vec_corp.fit(trainDF['text'])
count_vect_corp_mod.fit(trainDF['text'])
tfidf_vec_corp_mod.fit(trainDF['text'])

TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english',
                token_pattern='\\w{1,}')

In [18]:
results = pd.DataFrame()

list_of_vectorizers = [count_vect_corp, tfidf_vec_corp, count_vect_corp_mod, tfidf_vec_corp_mod, count_vectorizer_stemmed, 
                       tfidf_vectorizer_stemmed, count_vectorizer_lemmatized, tfidf_vectorizer_lemmatized]

for i, vectorizer in enumerate(list_of_vectorizers):

    xtrain_count = vectorizer.transform(train_x)
    xvalid_count = vectorizer.transform(valid_x)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_count, train_y)
    predictions = classifier.predict(xvalid_count)
    results[i] = [vectorizer, accuracy_score(valid_y, predictions)]

In [19]:
results = results.T.rename(columns={0:'vectorizer', 1:'accuracy'})
results

,vectorizer,accuracy
0,"CountVectorizer(token_pattern='\\w{1,}')",0.8684
1,"TfidfVectorizer(token_pattern='\\w{1,}')",0.8628
2,"CountVectorizer(max_df=0.9, max_features=1000,...",0.8328
3,"TfidfVectorizer(max_df=0.9, max_features=1000,...",0.8356
4,"CountVectorizer(max_df=0.9, max_features=1000,...",0.7352
5,"TfidfVectorizer(max_df=0.9, max_features=1000,...",0.7492
6,"CountVectorizer(max_df=0.9, max_features=1000,...",0.7704
7,"TfidfVectorizer(max_df=0.9, max_features=1000,...",0.7808


* Самый лучший результат показывает CountVectorizer с настройками по дефолту и разбиением по паттерну, обученный на корпусе. При изменении min_df, max_df, удалении стоп-слов, уменьшении количества фичей - скор падает. 
* Обученные на твитах векторайзеры показывают более низкий результат. Из из них лучше всего работает TfidfVectorizer, обученнный на лемматизированных твитах. Хуже всего - CountVectorizer, обученный на tweet_stemmed.

**PCA для сокращения размерности**

In [20]:
values = tfidf_vec_corp.fit_transform(trainDF['text'])
print(values.shape)

(10000, 31681)


In [21]:
results_pca = pd.DataFrame()

n_components = [200, 500, 1000, 3000, 5000]

for i, num in enumerate(n_components):
    svd = TruncatedSVD(n_components=num)
    xtrain_count =  count_vect_corp.transform(train_x)
    xvalid_count =  count_vect_corp.transform(valid_x)

    xtrain_count_pca =  svd.fit_transform(xtrain_count)
    xvalid_count_pca =  svd.transform(xvalid_count)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_count_pca, train_y)
    predictions = classifier.predict(xvalid_count_pca)
    results_pca[i] = [num, accuracy_score(valid_y, predictions)]

In [22]:
results_pca = results_pca.T.rename(columns={0:'num_components', 1:'accuracy'})
results_pca

,num_components,accuracy
0,200.0,0.8312
1,500.0,0.8552
2,1000.0,0.8620
3,3000.0,0.8636
4,5000.0,0.8696


In [24]:
svd = TruncatedSVD(n_components=5000)
xtrain_count =  tfidf_vec_corp.transform(train_x)
xvalid_count =  tfidf_vec_corp.transform(valid_x)

xtrain_count_pca =  svd.fit_transform(xtrain_count)

xvalid_count_pca =  svd.transform(xvalid_count)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count_pca, train_y)
predictions = classifier.predict(xvalid_count_pca)
accuracy_score(valid_y, predictions)

0.8664

In [25]:
svd = TruncatedSVD(n_components=3000)
xtrain_count =  tfidf_vec_corp.transform(train_x)
xvalid_count =  tfidf_vec_corp.transform(valid_x)

xtrain_count_pca =  svd.fit_transform(xtrain_count)

xvalid_count_pca =  svd.transform(xvalid_count)

classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count_pca, train_y)
predictions = classifier.predict(xvalid_count_pca)
accuracy_score(valid_y, predictions)

0.868

Качество предсказаний выросло на 0,0012 при снижении размерности до 5000. Но при дальнейшем снижении размерности скор уменьшается.
Проверка на другом векторайзере также показвыет аналогичный результат: +0,0036. Однако при уменьшении размерности до 3000 качество для векторайзера tfidf_vec_corp еще выросло. 